In [1]:
%pip install -q ./arc/arcadia/dataforge/

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from crowdom import bots

import yaml
import logging.config

In [3]:
with open('logging.yaml') as f:
    logging.config.dictConfig(yaml.full_load(f.read()))

In [4]:
register = bots.NDARegisterClient()

In [8]:
from datetime import timedelta

from crowdom import base, client, datasource, objects

In [10]:
annotation_function = base.AnnotationFunction(
    inputs=(objects.Audio,),
    outputs=(objects.Text,)
)

In [11]:
instruction = {
    'RU': 'Запишите звучащие на аудио слова, без знаков препинания и заглавных букв.',
    'EN': 'Transcribe the audio, without any punctuation or capitalization.'}

In [12]:
task_spec = base.TaskSpec(
    id='audio-transcription',
    function=annotation_function,
    name={'EN': 'Audio transcription', 'RU': 'Расшифровка аудио'},
    description={'EN': 'Transcribe short audios', 'RU': 'Расшифровка коротких аудио'},
    instruction=instruction)

In [13]:
lang = 'EN'

In [14]:
task_spec_en = client.AnnotationTaskSpec(task_spec, lang)

In [17]:
solved_tasks = datasource.read_tasks(
    'bots_tasks.json',
    task_spec_en.task_mapping,
    has_solutions=True,
)

In [18]:
# audios are 3-10 seconds each, and workers need time to transcribe them
task_duration_hint = timedelta(seconds=10)

# Annotation

In [19]:
# todo - пока контрольные задания только из ответа OK для ботов могут быть логичные

In [21]:
from random import shuffle

In [22]:
shuffle(solved_tasks)

In [23]:
solutions = bots.create_tasks_info_dict(
    task_spec_en,
    solved_tasks,
    {
        bots.TaskComplexity.EASY: 624,
        bots.TaskComplexity.OK: 120,
        bots.TaskComplexity.HARD: 80,
    }
)

In [26]:
markup_pool_id = ...
check_pool_id = ...

In [27]:
worker_types = [
    (bots.AnnotationWorkerConfig(bots.WorkerSpeed.MEDIUM, 0.85), 2),
    (bots.AnnotationWorkerConfig(bots.WorkerSpeed.SLOW, 0.9), 2),
    (bots.AnnotationWorkerConfig(bots.WorkerSpeed.FAST, 0.7), 4),
    (bots.AnnotationWorkerConfig(bots.WorkerSpeed.FAST, 0.5), 4),
]

In [28]:
check_worker_types = [
    (bots.AnnotationCheckWorkerConfig(bots.WorkerSpeed.SLOW, 0.85), 2),
    (bots.AnnotationCheckWorkerConfig(bots.WorkerSpeed.MEDIUM, 0.9), 2),
    (bots.AnnotationCheckWorkerConfig(bots.WorkerSpeed.FAST, 0.7), 4),
    (bots.AnnotationCheckWorkerConfig(bots.WorkerSpeed.FAST, 0.5), 4),
]

In [ ]:
wg_markup = bots.WorkerGroup(
    markup_pool_id,
    task_spec_en,
    task_duration_hint,
    worker_types,
    solutions)

In [ ]:
wg_check = bots.WorkerGroup(
    check_pool_id,
    task_spec_en.check,
    task_duration_hint,
    check_worker_types,
    solutions)

In [ ]:
for wg in [wg_markup, wg_check]:
    wg.start()

In [ ]:
# after launch_annotation finishes:

In [35]:
for wg in [wg_markup, wg_check]:
    wg.stop()
